In [1]:
import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class GumtreeApartmentsSpider(scrapy.Spider):
    name = 'gumtreeapartmentsspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
        ]
    found_apartments = []
   
    custom_settings = {
        'DOWNLOAD_DELAY': '4.0', #obejście Fail2Ban
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"})
        item_urls = []
        next_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"})
            for h in hrefs:
                item_urls.append(self.top_url + h["href"]) #dopisuje 'https://www.gumtree.pl', bo otrzymany adres jest względny, przygotowuje listę wszystkich linków
            
        nexts = soup.findAll("a", {"class": "next"})
        
        for h in nexts:
            for h in hrefs:
                next_urls.append(self.top_url + h["href"])
        
        print (next_urls)
            
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item)

        for next_url in next_urls: #odwiedzanie następnych stron z linkami
            yield scrapy.Request(item_url, self.parse)
        
        
    def parse_item(self, response): #item_url - odwiedzanie strony, #self.parse_item - przetworzenie przy pomocy funkcji
        self.logger.info('Got successful response from {}'.format(response.url))
        found_apartments = []
        soup = BeautifulSoup(response.text, 'html.parser') 
        apartments = soup.find('div', {'class': 'vip-header-and-details'})
        apartment_details = dict()
        apartment_details['Nazwa ogłoszenia'] = apartments.find('span', class_ = 'myAdTitle').text
        apartment_details['Cena'] = apartments.find('span', class_ = 'amount').text.replace("\xa0"," ") #replace w celu usunięcia twardej spacji
        container = soup.find('ul', class_ = 'selMenu') #zebranie informacji z ramki do kontenera
        
        nazwy = container.findAll('span', class_ = 'name')
        szczegoly = container.findAll('span', class_ = 'value')
        apartment_details.update({name.text: value.text.strip() for name, value in zip(nazwy, szczegoly)}) #dodawanie elementów z ramki do listy i usunięcie whitespace'ów
        
        yield found_apartments.append(apartment_details)
        print(found_apartments)

In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(GumtreeApartmentsSpider)
process.start()

2019-04-12 15:34:02 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-12 15:34:02 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.4.0-17134-Microsoft-x86_64-with-Ubuntu-18.04-bionic
2019-04-12 15:34:02 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': '4.0', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-04-12 15:34:02 [scrapy.extensions.telnet] INFO: Telnet Password: 18ea3128621ec6c4
2019-04-12 15:34:02 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-04-12 15:34:02 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloade

['https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/mieszkanie-+-warszawa-mokotow-sluzewiec-bukowinska/1004746848900911115501509']


2019-04-12 15:34:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/nowoczesny-apartament-na-gornym-mokotowie/1004703335820910564665709> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-12 15:34:09 [gumtreeapartmentsspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/nowoczesny-apartament-na-gornym-mokotowie/1004703335820910564665709


[{'Nazwa ogłoszenia': 'Nowoczesny apartament na Górnym Mokotowie', 'Cena': '780 000 zł', 'Data dodania': '07/04/2019', 'Lokalizacja': 'Mokotów, \n    \n    \n        Warszawa', 'Na sprzedaż przez': 'Właściciel', 'Rodzaj nieruchomości': 'Mieszkanie', 'Liczba pokoi': '2 pokoje', 'Liczba łazienek': '1 łazienka', 'Wielkość (m2)': '50', 'Parking': 'Garaż'}]


2019-04-12 15:34:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/mieszkanie-+-warszawa-mokotow-sluzewiec-bukowinska/1004746848900911115501509> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-12 15:34:14 [gumtreeapartmentsspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/mieszkanie-+-warszawa-mokotow-sluzewiec-bukowinska/1004746848900911115501509


...

[{'Nazwa ogłoszenia': 'Mieszkanie - Warszawa > Śródmieście > al. Jana Pawła II', 'Cena': '450 000 zł', 'Data dodania': '12/04/2019', 'Lokalizacja': 'Śródmieście, \n    \n    \n        Warszawa', 'Na sprzedaż przez': 'Agencja', 'Rodzaj nieruchomości': 'Mieszkanie', 'Liczba pokoi': '2 pokoje', 'Wielkość (m2)': '40'}]


2019-04-12 15:34:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/mieszkanie-+-warszawa-praga+poludnie-saska-kepa-al-stanow-zjednoczonych/1004746610990911115501509> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-12 15:34:47 [gumtreeapartmentsspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/praga-poludnie/mieszkanie-+-warszawa-praga+poludnie-saska-kepa-al-stanow-zjednoczonych/1004746610990911115501509


[{'Nazwa ogłoszenia': 'Mieszkanie - Warszawa > Praga-Południe > Saska Kępa > al. Stanów Zjednoczonych', 'Cena': '430 000 zł', 'Data dodania': '12/04/2019', 'Lokalizacja': 'Praga Południe, \n    \n    \n        Warszawa', 'Na sprzedaż przez': 'Agencja', 'Rodzaj nieruchomości': 'Mieszkanie', 'Liczba pokoi': '2 pokoje', 'Liczba łazienek': '1 łazienka', 'Wielkość (m2)': '50'}]


2019-04-12 15:34:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/3-pokoje-na-tagowku-ul-turmoncka/1004747632120911378860309> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-12 15:34:52 [gumtreeapartmentsspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/targowek/3-pokoje-na-tagowku-ul-turmoncka/1004747632120911378860309


[{'Nazwa ogłoszenia': '3 pokoje na Tagówku ul. Turmoncka', 'Cena': '439 000 zł', 'Data dodania': '12/04/2019', 'Lokalizacja': 'Targówek, \n    \n    \n        Warszawa', 'Na sprzedaż przez': 'Agencja', 'Rodzaj nieruchomości': 'Mieszkanie', 'Liczba pokoi': '3 pokoje', 'Liczba łazienek': '1 łazienka', 'Wielkość (m2)': '62'}]
